In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
os.environ.get('CUDA_VISIBLE_DEVICES',None)

'1'

In [4]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input

In [5]:
from tensorflow.keras import Model

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [7]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [8]:
regex = re.compile('[a-zA-Z ]')
clean_text = lambda text: re.sub('[^a-zA-Z ]', '' ,text)
joined_text = lambda text : ' '.join(text)

In [9]:
df_train = df_train.drop(columns='id')
df_train.head(1)

,prompt,A,B,C,D,E,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D


In [10]:
features = df_train.iloc[:,:-1]
label =  df_train['answer']

In [11]:
encoded_label = encoder.fit_transform(label)

In [12]:
y_tarin = to_categorical(encoded_label,num_classes=5)

In [13]:
cleaned_features = features.applymap(clean_text)

In [14]:
combined_features = cleaned_features.prompt+cleaned_features.A+cleaned_features.B+cleaned_features.C+cleaned_features.D+cleaned_features.E

In [15]:

len(combined_features)

200

In [16]:
max_len = max(len(i) for i in combined_features)
max_len

2394

In [17]:
max_vocab = 10000

In [18]:
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(combined_features)
sequences = tokenizer.texts_to_sequences(combined_features)
padded_sequences = pad_sequences(sequences,maxlen =max_len,padding='post')

In [19]:
model = Sequential()

In [20]:
model.add(Embedding(input_dim=max_vocab,output_dim=20,input_length=max_len))
model.add(LSTM(25,activation='tanh'))
model.add(Dense(5,activation='softmax'))

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(padded_sequences,y_tarin,validation_split=0.2,epochs=25)

Epoch 1/25
5/5 [==============================] - 3s 506ms/step - loss: 1.6116 - accuracy: 0.1750 - val_loss: 1.6089 - val_accuracy: 0.3000
Epoch 2/25
5/5 [==============================] - 2s 421ms/step - loss: 1.6091 - accuracy: 0.2250 - val_loss: 1.6051 - val_accuracy: 0.3000
Epoch 3/25
5/5 [==============================] - 2s 424ms/step - loss: 1.6085 - accuracy: 0.2250 - val_loss: 1.6014 - val_accuracy: 0.3000
Epoch 4/25
5/5 [==============================] - 2s 430ms/step - loss: 1.6071 - accuracy: 0.2250 - val_loss: 1.5989 - val_accuracy: 0.3000
Epoch 5/25
5/5 [==============================] - 2s 424ms/step - loss: 1.6065 - accuracy: 0.2250 - val_loss: 1.5962 - val_accuracy: 0.3000
Epoch 6/25
5/5 [==============================] - 2s 422ms/step - loss: 1.6058 - accuracy: 0.2250 - val_loss: 1.5942 - val_accuracy: 0.3000
Epoch 7/25
5/5 [==============================] - 2s 422ms/step - loss: 1.6056 - accuracy: 0.2250 - val_loss: 1.5924 - val_accuracy: 0.3000
Epoch 8/25
5/5 [====